# Feed Forward Neural Network SMS Text Classifier

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called predict_message that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the SMS Spam Collection dataset. The dataset has already been grouped into train data and test data.
You're will use a Keras Sequential Model, which is a type of feedforward neural network (FFNN). It’s called Sequential because each layer feeds into the next in a straight line — no branches or loops.


# Libraries

In [6]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.5/600.5 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 75.5 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires ml-dtypes<0.5.0,>=0.4.0, but you have ml-dtypes 0.5.1 which is incompatible.
2.20.0-dev20250508


# Dataset

In [7]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2025-05-08 23:19:04--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.1’

train-data.tsv.1    100%[===================>] 349.84K  --.-KB/s    in 0.03s   

2025-05-08 23:19:05 (10.0 MB/s) - ‘train-data.tsv.1’ saved [358233/358233]

--2025-05-08 23:19:05--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.1’

valid-data.tsv.1    100%[========

# Feature Engineering

In [8]:
# Load and preprocess the dataset
train_data = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
test_data = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])

# Convert labels to binary values
train_data['label'] = train_data['label'].map({'ham': 0, 'spam': 1})
test_data['label'] = test_data['label'].map({'ham': 0, 'spam': 1})

# Text vectorization
max_vocab_size = 10000
max_sequence_length = 100

vectorizer = tf.keras.layers.TextVectorization(max_tokens=max_vocab_size, output_sequence_length=max_sequence_length)
vectorizer.adapt(train_data['message'])

# Prepare training and validation data
X_train = vectorizer(train_data['message']).numpy()
y_train = train_data['label'].values

X_test = vectorizer(test_data['message']).numpy()
y_test = test_data['label'].values

# Model Building and Traning

In [14]:
# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(max_vocab_size, 32),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0.3)]
)

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)

Epoch 1/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - binary_accuracy: 0.6615 - loss: 0.4459 - val_binary_accuracy: 0.8635 - val_loss: 0.3679
Epoch 2/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - binary_accuracy: 0.8610 - loss: 0.3706 - val_binary_accuracy: 0.8635 - val_loss: 0.3601
Epoch 3/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - binary_accuracy: 0.8650 - loss: 0.3500 - val_binary_accuracy: 0.8628 - val_loss: 0.3420
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - binary_accuracy: 0.8653 - loss: 0.3371 - val_binary_accuracy: 0.9138 - val_loss: 0.2854
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - binary_accuracy: 0.9284 - loss: 0.2590 - val_binary_accuracy: 0.9677 - val_loss: 0.1767
Epoch 6/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - binary_accuracy: 0.9736 - loss: 0.1347 - val_binary_accuracy: 0.9727 - val_loss: 0.0943
Epoch 7/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - binary_accuracy: 0.9833 - loss: 0.0764 - val_binary_accuracy: 0.9749 - val_loss: 0.0841
Epoch

# Predict Message Function

In [15]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])

def predict_message(pred_text):
    pred_vector = vectorizer([pred_text])
    pred_prob = model.predict(pred_vector)[0, 0]
    pred_label = 'ham' if pred_prob < 0.2 else 'spam'
    return [pred_prob, pred_label]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
[np.float32(0.0026396962), 'ham']


# Test

In [17]:
def test_predictions():
    test_messages = [
        "how are you doing today",
        "sale today! to stop texts call 98912460324",
        "i dont want to go. can we try it a different day? available sat",
        "our new mobile video service is live. just install on your phone to start watching.",
        "you have won £1000 cash! call to claim your prize.",
        "i'll bring it tomorrow. don't forget the milk.",
        "wow, is your arm alright. that happened to me one time too"
    ]

    test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
    passed = True

    for msg, ans in zip(test_messages, test_answers):
        prediction = predict_message(msg)
        print(f"Message: {msg}\nPrediction: {prediction}\nExpected: {ans}\n")
        if prediction[1] != ans:
            passed = False

    if passed:
        print("You passed the challenge. Great job!")
    else:
        print("You haven't passed yet. Let's debug!")

test_predictions()



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Message: how are you doing today
Prediction: [np.float32(0.0026396962), 'ham']
Expected: ham

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Message: sale today! to stop texts call 98912460324
Prediction: [np.float32(0.26422602), 'spam']
Expected: spam

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Message: i dont want to go. can we try it a different day? available sat
Prediction: [np.float32(0.0018455419), 'ham']
Expected: ham

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Message: our new mobile video service is live. just install on your phone to start watching.
Prediction: [np.float32(0.8988877), 'spam']
Expected: spam

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Message: you have won £1000 cash! call to claim your prize.
Prediction: [np.float32(0.9431174), 'spam']
Expected: spam

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Message: i'll bring it tomorrow. don't forget the milk.
Prediction: [np.float32(0.0014415048), 'ham']
Expected: ham

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Mes